## [PrivateScore 0.66]KoBERT+RoBERTa+KoElectra+KR-BERT

EDA를 통한 단어 전처리,

'monologg/kobert',  # KoBERT 모델

 'klue/roberta-base', # roberta 모델

 'klue/roberta-small',

 'klue/roberta-large',

 'monologg/koelectra-base-v3-discriminator',  # koelectra 모델

 "snunlp/KR-Medium" # KR-BERT 모델

위 모델들을 앙상블하여

validation dataset에 대한 f1 score를 가중치로 한 Soft voting을 수행하였습니다.

In [72]:
# BERT 모델들만 사용
model_names = [
    'klue/roberta-base',
    'klue/roberta-small', 
    'klue/roberta-large',
    'monologg/koelectra-base-v3-discriminator',
    "snunlp/KR-Medium"
]

In [ ]:
!{sys.executable} -m pip install nltk

In [ ]:
!{sys.executable} -m pip install gensim

In [ ]:
!{sys.executable} -m pip install sentencepiece

In [46]:
INFERENCE = False
TRAIN = not INFERENCE
print(f'Inference Mode : {INFERENCE}')
print(f'Train Mode : {TRAIN}')

Inference Mode : False
Train Mode : True


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

from types import SimpleNamespace

from eunjeon import Mecab

import torch.nn.functional as F
import re
import torch.optim as optim

from collections import Counter
import random

import os
import time
import pickle
import warnings
warnings.filterwarnings(action='ignore')
import gc

In [17]:
# seed_everything 함수 정의
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 설정
seed_everything(42)

In [88]:
# 설정
config = {
    "learning_rate": 5e-3,
    "epoch": 5,
    "batch_size": 64,
    "embedding_dim": 100,
    "max_len": 128,
    "patience":3,
    "lr_patience":2
}

CFG = SimpleNamespace(**config)

In [25]:
!{sys.executable} -m pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached jpype1-1.5.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (4.9 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
Using cached jpype1-1.5.2-cp312-cp312-macosx_10_9_universal2.whl (583 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [konlpy]2m2/3 [konlpy]


In [34]:
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs("한국어 형태소 분석을 시작합니다."))

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [29]:
from eunjeon import Mecab
mecab = Mecab()
print(mecab.morphs("한국어 형태소 분석을 시작합니다."))

Exception: The MeCab dictionary does not exist at "/Users/leeyoungho/miniforge3/envs/ai_3/lib/python3.12/site-packages/eunjeon/data/mecabrc". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [57]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

# 간단한 토큰화 예시
text = "한국어 형태소 분석을 시작합니다."
words = text.split()  # 간단한 공백 기준 분리
print(words)

['한국어', '형태소', '분석을', '시작합니다.']


In [38]:
# EDA
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

# '제목'과 '키워드' 결합
train_df['제목_키워드'] = train_df['제목'] + train_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + test_df['키워드']

In [40]:
# 기본 불용어 리스트
stop_words = [
    '이', '그', '저', '것', '수', '등', '및', '또는', '그리고', '하지만', '그런데',
    '그러나', '따라서', '그래서', '왜냐하면', '만약', '만일', '아니면', '또한',
    '이것', '저것', '그것', '무엇', '어떤', '어떻게', '언제', '어디서', '왜',
    '가', '이', '을', '를', '의', '에', '에서', '로', '으로', '와', '과',
    '도', '만', '부터', '까지'
]

# 파일로 저장
with open("stopwords.txt", "w", encoding="utf-8") as f:
    for word in stop_words:
        f.write(word + "\n")

print("stopwords.txt 파일이 생성되었습니다.")

stopwords.txt 파일이 생성되었습니다.


In [44]:
# 데이터 전처리에 필요한 불용어 리스트 작성 및 생성
with open("stopwords.txt",encoding = 'utf-8-sig') as f:
    stop_words = f.readlines()
stop_words = [word.strip() for word in stop_words]

# 특수문자 제거 및 간단한 전처리 함수
def clean_text(text):
    if pd.isna(text):
        return ""
    # 숫자 제거
    text = re.sub(r'\d+', '', str(text))
    # 특수문자 제거 (한글, 영문, 공백만 남김)
    text = re.sub(r'[^가-힣a-zA-Z\s]', '', str(text))
    # 연속된 공백을 하나로
    text = re.sub(r'\s+', ' ', str(text)).strip()
    return text

# 간단한 토큰화 함수 (형태소 분석 없이)
def simple_tokenize(text):
    if pd.isna(text):
        return []
    # 공백 기준으로 분리
    tokens = str(text).split()
    # 길이가 1보다 큰 토큰만 유지
    tokens = [token for token in tokens if len(token) > 1]
    return tokens

# 데이터 전처리
train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 텍스트 정리
train_df['제목_키워드'] = train_df['제목_키워드'].apply(clean_text)
test_df['제목_키워드'] = test_df['제목_키워드'].apply(clean_text)

# 간단한 토큰화 적용
train_df['tokens'] = train_df['제목_키워드'].apply(simple_tokenize)
test_df['tokens'] = test_df['제목_키워드'].apply(simple_tokenize)

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
label_decoder = {i: label for label, i in label_encoder.items()}
train_df['label'] = train_df['분류'].map(label_encoder)

print("전처리 완료!")
print(f"훈련 데이터 샘플: {train_df['제목_키워드'].iloc[0]}")
print(f"토큰 샘플: {train_df['tokens'].iloc[0]}")
print(f"레이블 개수: {len(label_encoder)}")

전처리 완료!
훈련 데이터 샘플: 용인문화재단 인문학 콘서트 당신이 모르는 뮤지컬 이야기 개최 용인문화재단인문학콘서트뮤지컬이야기개최인문학콘서트뮤지컬이야기용인문화재단용인문화재단제공시민들심신여유안정인문학콘서트개최재단일큰어울마당용인시평생학습관인문학콘서트뮤지컬이야기인문학콘서트방송매체왕성활동뮤지컬평론가교수원종원순천향대진행이건명김소향김보경유건우배우들정상급뮤지컬배우라이브무대동시만끽공연공연홍지민뮤지컬배우참여라이브무대선사영화뮤지컬합성어무비컬주제맘마미아드림걸즈유명작품교수위트해설예정공연초등학생관람초등학생가능티켓가격전석공연용인문화재단누리집확인
토큰 샘플: ['용인문화재단', '인문학', '콘서트', '당신이', '모르는', '뮤지컬', '이야기', '개최', '용인문화재단인문학콘서트뮤지컬이야기개최인문학콘서트뮤지컬이야기용인문화재단용인문화재단제공시민들심신여유안정인문학콘서트개최재단일큰어울마당용인시평생학습관인문학콘서트뮤지컬이야기인문학콘서트방송매체왕성활동뮤지컬평론가교수원종원순천향대진행이건명김소향김보경유건우배우들정상급뮤지컬배우라이브무대동시만끽공연공연홍지민뮤지컬배우참여라이브무대선사영화뮤지컬합성어무비컬주제맘마미아드림걸즈유명작품교수위트해설예정공연초등학생관람초등학생가능티켓가격전석공연용인문화재단누리집확인']
레이블 개수: 56


In [48]:
if TRAIN:
    # jar 디렉토리가 없으면 생성
    os.makedirs('./jar', exist_ok=True)
    
    # 레이블 인코더 저장
    with open('./jar/label_encoder.pkl', 'wb') as f:
        pickle.dump(label_encoder, f)

    # 레이블 디코더 저장
    with open('./jar/label_decoder.pkl', 'wb') as f:
        pickle.dump(label_decoder, f)

In [49]:
if INFERENCE:
    # 레이블 인코더 불러오기
    with open('./jar/label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)

    # 레이블 디코더 불러오기
    with open('./jar/label_decoder.pkl', 'rb') as f:
        label_decoder = pickle.load(f)

In [52]:
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')
# 동의어 찾기
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Synonym Replacement (동의어 교체)
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set(new_words))
    random.shuffle(random_word_list)
    
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:  # 교체된 단어 수 제한
            break

    return new_words

# Random Insertion (무작위 삽입)
def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:  # 동의어를 찾을 수 없을 경우 중단
            return
    random_synonym = random.choice(synonyms)
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

# Random Swap (무작위 교환)
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def swap_word(new_words):
    if len(new_words) < 2:
        return new_words
    idx1, idx2 = random.sample(range(len(new_words)), 2)
    new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    return new_words

# Random Deletion (무작위 삭제)
def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:  # 확률적으로 삭제
            new_words.append(word)

    if len(new_words) == 0:  # 모든 단어가 삭제되면 한 단어를 남김
        return [words[random.randint(0, len(words)-1)]]

    return new_words

# 증강 기법을 적용하여 여러 증강 텍스트 생성
def augment_text(words, num_aug=4):
    augmented_texts = []
    
    num_words = len(words)
    n_sr = max(1, int(0.1 * num_words))  # Synonym Replacement 개수
    n_ri = max(1, int(0.1 * num_words))  # Random Insertion 개수
    n_rs = max(1, int(0.1 * num_words))  # Random Swap 개수
    p_rd = 0.1  # Random Deletion 확률

    augmented_texts.append(synonym_replacement(words, n_sr))  # 동의어 교체
    augmented_texts.append(random_insertion(words, n_ri))  # 무작위 삽입
    augmented_texts.append(random_swap(words, n_rs))  # 무작위 교환
    augmented_texts.append(random_deletion(words, p_rd))  # 무작위 삭제
    
    return augmented_texts

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leeyoungho/nltk_data...


In [53]:
# 각 클래스의 데이터 수 확인
class_counts = train_df['label'].value_counts()
print("각 클래스의 데이터 수:", class_counts)

# 최대 클래스 데이터 수 확인 (가장 많은 데이터가 있는 클래스)
max_class_count = class_counts.max()

# 2. 증강된 데이터를 저장할 리스트
augmented_texts = []
augmented_labels = []

# 3. 각 클래스의 데이터 수를 최대값에 맞추기 위해 증강 적용
for label in class_counts.index:
    class_data = train_df[train_df['label'] == label]
    current_class_count = class_counts[label]
    
    # 데이터 증강이 필요한 수만큼 반복
    while current_class_count < max_class_count:
        for tokens in class_data['tokens']:
            # 증강된 데이터 생성
            new_texts = augment_text(tokens)
            
            # 최대 클래스 수를 초과하지 않도록 조정
            for new_text in new_texts:
                augmented_texts.append(new_text)
                augmented_labels.append(label)
                current_class_count += 1
                if current_class_count >= max_class_count:
                    break
            if current_class_count >= max_class_count:
                break

각 클래스의 데이터 수: label
1     26950
10     3454
21     2568
33     2318
18     1480
11      995
7       966
5       950
20      845
22      841
25      711
0       671
36      640
16      625
14      621
26      617
3       608
27      589
28      537
29      536
15      447
23      396
9       387
37      375
6       349
2       337
31      335
8       328
4       327
43      279
44      248
32      243
13      238
45      229
17      221
40      215
19      213
51      202
42      190
34      178
35      173
38      160
12      128
30      124
47      113
46      110
48      102
52       94
54       90
39       76
49       67
24       31
55       29
53       27
50       17
41        9
Name: count, dtype: int64


In [54]:
# 4. 증강된 데이터프레임 생성
augmented_df = pd.DataFrame({'tokens': augmented_texts, 'label': augmented_labels})

# 5. 기존 데이터에 증강된 데이터 추가
train_df = pd.concat([train_df, augmented_df], ignore_index=True)

# 클래스별로 데이터 수 확인
print("균형 맞춘 후 각 클래스의 데이터 수:", train_df['label'].value_counts())

균형 맞춘 후 각 클래스의 데이터 수: label
0     26950
1     26950
30    26950
31    26950
32    26950
33    26950
34    26950
35    26950
36    26950
37    26950
38    26950
39    26950
40    26950
41    26950
42    26950
43    26950
44    26950
45    26950
46    26950
47    26950
48    26950
49    26950
50    26950
51    26950
52    26950
53    26950
54    26950
29    26950
28    26950
27    26950
13    26950
2     26950
3     26950
4     26950
5     26950
6     26950
7     26950
8     26950
9     26950
10    26950
11    26950
12    26950
14    26950
26    26950
15    26950
16    26950
17    26950
18    26950
19    26950
20    26950
21    26950
22    26950
23    26950
24    26950
25    26950
55    26950
Name: count, dtype: int64


### Word2Vec 사용 안함!!!!

In [59]:
# 데이터 분할 (Word2Vec 없이)
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)
print(f"훈련 데이터: {len(train_df)}, 검증 데이터: {len(val_df)}")

# 데이터셋 클래스 정의 (BERT 모델용)
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 데이터로더 생성 함수
def create_data_loader(texts, labels, tokenizer, batch_size=16, max_len=128):
    dataset = TextDataset(texts, labels, tokenizer, max_len)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

훈련 데이터: 1207360, 검증 데이터: 301840


In [60]:
# 데이터 분할
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

In [62]:
if TRAIN:
    # 데이터프레임을 피클로 저장
    with open('./jar/train_df.pkl', 'wb') as f:
        pickle.dump(train_df, f)
    with open('./jar/test_df.pkl', 'wb') as f:
        pickle.dump(test_df, f)
    with open('./jar/val_df.pkl', 'wb') as f:
        pickle.dump(val_df, f)
    
    print("데이터프레임들이 저장되었습니다.")

데이터프레임들이 저장되었습니다.


In [63]:
if INFERENCE:
    # 데이터프레임 불러오기
    with open('./jar/train_df.pkl', 'rb') as f:
        train_df = pickle.load(f)
    with open('./jar/test_df.pkl', 'rb') as f:
        test_df = pickle.load(f)
    with open('./jar/val_df.pkl', 'rb') as f:
        val_df = pickle.load(f)
    # 임베딩 매트릭스 불러오기
    with open('./jar/embedding_matrix.pkl', 'rb') as f:
        embedding_matrix = pickle.load(f)
    # word2idx 사전 불러오기
    with open('./jar/word2idx.pkl', 'rb') as f:
        word2idx = pickle.load(f)
    # idx2word 사전 불러오기
    with open('./jar/idx2word.pkl', 'rb') as f:
        idx2word = pickle.load(f)

In [64]:
# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer=None, word2idx=None, max_len=128, model_type='cnn'):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.word2idx = word2idx
        self.max_len = max_len
        self.model_type = model_type  # 'cnn', 'lstm' or 'bert'

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx] if self.labels is not None else -1

        if self.model_type == 'bert':
            encoding = self.tokenizer.encode_plus(
                " ".join(text),
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=False,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt',
            )
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long)
            }
        elif self.model_type in ['cnn', 'lstm']:
            tokens = text
            seq = [self.word2idx.get(token, self.word2idx["<UNK>"]) for token in tokens]
            if len(seq) < self.max_len:
                seq += [self.word2idx["<PAD>"]] * (self.max_len - len(seq))
            else:
                seq = seq[:self.max_len]
            seq = torch.tensor(seq, dtype=torch.long)
            label = torch.tensor(label, dtype=torch.long)
            return {
                'input_ids': seq,
                'labels': label
            }
        else:
            raise ValueError("Invalid model_type. Choose 'bert', 'cnn', or 'lstm'.")

In [65]:
# 새로운 모델들과 토크나이저 로드
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_names = [
    'monologg/kobert',  # KoBERT 모델
    'klue/roberta-base', # roberta 모델
    'klue/roberta-small',
    'klue/roberta-large',
    'monologg/koelectra-base-v3-discriminator',  # koelectra 모델
    "snunlp/KR-Medium" # KR-BERT 모델
]

models = {}
tokenizers = {}

In [68]:
for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(label_encoder),
        trust_remote_code=True
    ).to('mps')
    tokenizers[model_name] = tokenizer
    models[model_name] = model

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/408M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-Medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# BERT 모델들만 사용
from torch.utils.data import DataLoader

train_loaders = {}
val_loaders = {}
test_loaders = {}

# 배치 크기 설정 (모든 모델에 대해 동일하게)
batch_size = CFG.batch_size

for model_name in model_names:
    tokenizer = tokenizers[model_name]
    
    # 데이터셋 생성
    train_dataset = TextDataset(train_df['tokens'].tolist(), train_df['label'].tolist(), tokenizer=tokenizer, max_len=CFG.max_len, model_type='bert')
    val_dataset = TextDataset(val_df['tokens'].tolist(), val_df['label'].tolist(), tokenizer=tokenizer, max_len=CFG.max_len, model_type='bert')
    test_dataset = TextDataset(test_df['tokens'].tolist(), None, tokenizer=tokenizer, max_len=CFG.max_len, model_type='bert')

    # 데이터로더 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    train_loaders[model_name] = train_loader
    val_loaders[model_name] = val_loader
    test_loaders[model_name] = test_loader

print("BERT 모델들의 데이터로더가 생성되었습니다.")

BERT 모델들의 데이터로더가 생성되었습니다.


In [93]:
# 테스트용으로 데이터 크기 줄이기
train_df_small = train_df.sample(n=10000, random_state=42)  # 1만개만 사용
val_df_small = val_df.sample(n=2000, random_state=42)      # 2천개만 사용

print(f"원본 훈련 데이터: {len(train_df)}")
print(f"줄인 훈련 데이터: {len(train_df_small)}")
print(f"원본 검증 데이터: {len(val_df)}")
print(f"줄인 검증 데이터: {len(val_df_small)}")

# 작은 데이터로 데이터로더 다시 생성
train_loaders = {}
val_loaders = {}
test_loaders = {}

batch_size = CFG.batch_size  # 배치 크기도 늘리기
print(f"배치 크기: {batch_size}")

for model_name in model_names:
    tokenizer = tokenizers[model_name]
    
    # 작은 데이터셋으로 생성
    train_dataset = TextDataset(train_df_small['tokens'].tolist(), train_df_small['label'].tolist(), tokenizer=tokenizer, max_len=CFG.max_len, model_type='bert')
    val_dataset = TextDataset(val_df_small['tokens'].tolist(), val_df_small['label'].tolist(), tokenizer=tokenizer, max_len=CFG.max_len, model_type='bert')
    test_dataset = TextDataset(test_df['tokens'].tolist(), None, tokenizer=tokenizer, max_len=CFG.max_len, model_type='bert')

    # 데이터로더 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    train_loaders[model_name] = train_loader
    val_loaders[model_name] = val_loader
    test_loaders[model_name] = test_loader

print("작은 데이터로 데이터로더가 다시 생성되었습니다.")

원본 훈련 데이터: 965888
줄인 훈련 데이터: 10000
원본 검증 데이터: 241472
줄인 검증 데이터: 2000
배치 크기: 64
작은 데이터로 데이터로더가 다시 생성되었습니다.


In [94]:
# BERT 모델들만 사용하므로 모델 초기화 단순화
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 손실 함수 정의 (BERT 모델들에 공통으로 사용)
criterion = nn.CrossEntropyLoss()

# BERT 모델들의 옵티마이저들을 저장할 딕셔너리
optimizers = {}

# 각 BERT 모델에 대해 옵티마이저 생성
for model_name in model_names:
    optimizer = optim.AdamW(models[model_name].parameters(), lr=CFG.learning_rate)
    optimizers[model_name] = optimizer

print(f"BERT 모델 {len(model_names)}개의 옵티마이저가 생성되었습니다.")

사용 중인 디바이스: mps
BERT 모델 5개의 옵티마이저가 생성되었습니다.


In [100]:
# 학습 함수 정의 (BERT 모델들만 사용하므로 단순화)
from torch.optim import AdamW
import os

def train_and_evaluate(model, train_loader, val_loader, model_name):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=CFG.lr_patience)
    best_val_f1 = 0
    patience_counter = 0
    print(f"device: {device}")
    print(f"=== {model_name} 모델 학습 시작 ===")
    for epoch in range(CFG.epoch):
        model.train()
        total_loss = 0

        # Training loop
        for batch in tqdm(train_loader, desc=f'{model_name} Epoch {epoch + 1}/{CFG.epoch}'):
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"{model_name} - Epoch {epoch + 1} Training Loss: {avg_train_loss:.4f}")

        # Validation
        model.eval()
        val_predictions = []
        val_true_labels = []
        val_total_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)
                val_total_loss += loss.item()

                _, preds = torch.max(outputs.logits, dim=1)
                val_predictions.extend(preds.cpu().tolist())
                val_true_labels.extend(labels.cpu().tolist())

        avg_val_loss = val_total_loss / len(val_loader)
        print(f"{model_name} - Epoch {epoch + 1} Validation Loss: {avg_val_loss:.4f}")

        # 검증 결과 출력
        val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
        print(f"{model_name} - Epoch {epoch + 1} Validation F1 Score: {val_f1:.4f}")

        # Learning Rate Scheduler 업데이트
        scheduler.step(val_f1)
        
        # Early Stopping Check
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            patience_counter = 0
            # models 디렉토리 생성 후 모델 저장
            os.makedirs('./models', exist_ok=True)
            torch.save(model.state_dict(), f'./models/best_{model_name.replace("/", "_")}_model.pt')
            print("best model saved")
        else:
            patience_counter += 1
            if patience_counter >= CFG.patience:
                print(f"{model_name} Early stopping at epoch {epoch + 1}")
                break

    return best_val_f1

In [76]:
if INFERENCE:
    with open('./jar/val_f1_scores.pkl','rb') as f:
        val_f1_scores = pickle.load(f)

In [77]:
if TRAIN:
    val_f1_scores = {}

In [78]:
print(val_f1_scores)

{}


In [101]:
# 모델 학습 실행
val_f1_scores = {}

for model_name in model_names:
    print(f"\n=== {model_name} 모델 학습 시작 ===")
    
    model = models[model_name]
    train_loader = train_loaders[model_name]
    val_loader = val_loaders[model_name]
    
    # 모델 학습
    best_f1 = train_and_evaluate(model, train_loader, val_loader, model_name)
    val_f1_scores[model_name] = best_f1
    
    print(f"{model_name} 최고 F1 점수: {best_f1:.4f}")

print("\n=== 모든 모델 학습 완료 ===")
print("각 모델의 F1 점수:")
for model_name, f1_score in val_f1_scores.items():
    print(f"{model_name}: {f1_score:.4f}")


=== klue/roberta-base 모델 학습 시작 ===
device: mps
=== klue/roberta-base 모델 학습 시작 ===


klue/roberta-base Epoch 1/5: 100%|██████████| 157/157 [10:00<00:00,  3.83s/it]


klue/roberta-base - Epoch 1 Training Loss: 1.3134
klue/roberta-base - Epoch 1 Validation Loss: 1.3050
klue/roberta-base - Epoch 1 Validation F1 Score: 0.6445
best model saved


klue/roberta-base Epoch 2/5: 100%|██████████| 157/157 [10:41<00:00,  4.09s/it]


klue/roberta-base - Epoch 2 Training Loss: 1.0007
klue/roberta-base - Epoch 2 Validation Loss: 1.1874
klue/roberta-base - Epoch 2 Validation F1 Score: 0.6914
best model saved


...

klue/roberta-small - Epoch 4 Training Loss: 1.3588
klue/roberta-small - Epoch 4 Validation Loss: 1.4247
klue/roberta-small - Epoch 4 Validation F1 Score: 0.6389
best model saved


klue/roberta-small Epoch 5/5: 100%|██████████| 157/157 [06:33<00:00,  2.51s/it]


klue/roberta-small - Epoch 5 Training Loss: 1.0253
klue/roberta-small - Epoch 5 Validation Loss: 1.2638
klue/roberta-small - Epoch 5 Validation F1 Score: 0.6664
best model saved
klue/roberta-small 최고 F1 점수: 0.6664

=== klue/roberta-large 모델 학습 시작 ===
device: mps
=== klue/roberta-large 모델 학습 시작 ===


klue/roberta-large Epoch 1/5:   7%|▋         | 11/157 [31:49<7:02:23, 173.59s/it]


KeyboardInterrupt: 

In [102]:
# 다른 모델들 학습
if TRAIN:
    for model_name in model_names:
        print(f"=== {model_name} 모델 학습 시작 ===")
        model = models[model_name]
        model_path = f'./models/best_{model_name.replace("/", "_")}_model.pt'
        if os.path.exists(model_path):
            model.load_state_dict(torch.load(model_path, map_location=device))
        train_loader = train_loaders[model_name]
        val_loader = val_loaders[model_name]
        best_val_f1 = train_and_evaluate(model, train_loader, val_loader, model_name)
        val_f1_scores[model_name] = best_val_f1
        del model, model_path, train_loader, val_loader
        # macOS에서는 torch.cuda.empty_cache() 대신 gc.collect()만 사용
        gc.collect()

=== klue/roberta-base 모델 학습 시작 ===
device: mps
=== klue/roberta-base 모델 학습 시작 ===


klue/roberta-base Epoch 1/5:   0%|          | 0/157 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [103]:
# 테스트 세트 추론 (더 간단한 버전)
test_probs = {}

def predict_test(model, test_loader, model_name):
    model.eval()
    probs_list = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f'{model_name} Testing'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)
            probs_list.append(probs.cpu())

    test_probs[model_name] = torch.cat(probs_list, dim=0)
    print(f"{model_name} 예측 완료")

In [104]:
# 다른 모델들 테스트 추론 (안전한 버전)
for model_name in model_names:
    print(f"=== {model_name} 테스트 세트 추론 시작 ===")
    model = models[model_name]
    test_loader = test_loaders[model_name]
    
    # 저장된 최적의 모델 로드 (안전하게)
    model_path = f'./models/best_{model_name.replace("/", "_")}_model.pt'
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"모델 로드 완료: {model_path}")
    else:
        print(f"모델 파일이 없습니다: {model_path}")
        continue
    
    predict_test(model, test_loader, model_name)
    del model
    gc.collect()

=== klue/roberta-base 테스트 세트 추론 시작 ===
모델 로드 완료: ./models/best_klue_roberta-base_model.pt


klue/roberta-base Testing: 100%|██████████| 366/366 [02:51<00:00,  2.14it/s]


klue/roberta-base 예측 완료
=== klue/roberta-small 테스트 세트 추론 시작 ===
모델 로드 완료: ./models/best_klue_roberta-small_model.pt


klue/roberta-small Testing: 100%|██████████| 366/366 [01:29<00:00,  4.08it/s]


klue/roberta-small 예측 완료
=== klue/roberta-large 테스트 세트 추론 시작 ===
모델 파일이 없습니다: ./models/best_klue_roberta-large_model.pt
=== monologg/koelectra-base-v3-discriminator 테스트 세트 추론 시작 ===
모델 파일이 없습니다: ./models/best_monologg_koelectra-base-v3-discriminator_model.pt
=== snunlp/KR-Medium 테스트 세트 추론 시작 ===
모델 파일이 없습니다: ./models/best_snunlp_KR-Medium_model.pt


In [105]:
print(val_f1_scores)

{'klue/roberta-base': 0.7533415056626291, 'klue/roberta-small': 0.6664335882279931}


In [106]:
# 앙상블 적용
# 가중치 계산 (검증 F1 점수 기반)
if val_f1_scores:  # val_f1_scores가 비어있지 않은지 확인
    total_f1 = sum(val_f1_scores.values())
    model_weights = {model_name: val_f1_scores[model_name] / total_f1 for model_name in val_f1_scores.keys()}
    
    print("모델별 가중치:")
    for model_name, weight in model_weights.items():
        print(f"{model_name}: {weight:.4f}")
    
    # 앙상블 확률 계산
    ensemble_probs = None
    
    for model_name, probs in test_probs.items():
        if model_name in model_weights:  # 가중치가 있는 모델만 사용
            weight = model_weights[model_name]
            if ensemble_probs is None:
                ensemble_probs = weight * probs
            else:
                ensemble_probs += weight * probs
    
    # 최종 예측
    if ensemble_probs is not None:
        ensemble_preds = torch.argmax(ensemble_probs, dim=1)
        print(f"앙상블 예측 완료: {len(ensemble_preds)}개 샘플")
    else:
        print("앙상블 예측 실패: 예측할 모델이 없습니다.")
else:
    print("val_f1_scores가 비어있습니다. 모델 학습을 먼저 완료해주세요.")

모델별 가중치:
klue/roberta-base: 0.5306
klue/roberta-small: 0.4694
앙상블 예측 완료: 23405개 샘플


In [ ]:
with open('./jar/val_f1_scores.pkl','wb') as f:
    pickle.dump(val_f1_scores,f)
with open('./jar/test_probs.pkl', 'wb') as f:
    pickle.dump(test_probs,f)

In [107]:
# 라벨 디코딩
decoded_predictions_ensemble = [label_decoder[pred] for pred in ensemble_preds.tolist()]

In [109]:
# 제출 파일 생성
complete_time = time.strftime('%Y-%m-%d', time.localtime(time.time()))
sample_submission = pd.read_csv("./data/sample_submission.csv")  # 경로 수정
sample_submission["분류"] = decoded_predictions_ensemble
sample_submission.to_csv(f"./ensemble_submission_{complete_time}.csv", encoding='UTF-8-sig', index=False)